# LightGBM Champion - Vision 2026
## Configuration de Performance Maximale

Ce notebook represente l'etat de l'art de notre modelisation pour la Pitie-Salpetriere.
Strategie :
1. Signal de Fourier (Harmoniques 1-3) pour la cyclicite.
2. High-Capacity LightGBM (255 feuilles) pour capturer les nuances des flux.
3. Absence totale d'emojis pour la robustesse du code.

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration des donnees
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
dd = df_adm.groupby('date_entree').size().rename('admissions').asfreq('D', fill_value=0)

def create_champion_features(df_ts):
    df = pd.DataFrame(index=df_ts.index)
    df['admissions'] = df_ts.values
    
    # Signal de Fourier
    for k in range(1, 4):
        df[f'f_sin_{k}'] = np.sin(2 * np.pi * k * df.index.dayofyear / 365.25)
        df[f'f_cos_{k}'] = np.cos(2 * np.pi * k * df.index.dayofyear / 365.25)
    
    # Lags et Temporel
    df['day'] = df.index.dayofweek
    df['month'] = df.index.month
    for l in [1, 2, 7, 14]:
        df[f'lag_{l}'] = df['admissions'].shift(l)
        
    return df.dropna()

full_df = create_champion_features(dd)
X = full_df.drop('admissions', axis=1)
y = full_df['admissions']

X_train, X_test = X.iloc[:-30], X.iloc[-30:]
y_train, y_test = y.iloc[:-30], y.iloc[-30:]

In [ ]:
# Modele Champion
model = lgb.LGBMRegressor(
    objective='regression_l1', 
    n_estimators=5000, 
    learning_rate=0.005, 
    num_leaves=255, 
    random_state=42, 
    verbose=-1
)

print("Entrainement du modele Champion...")
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"MAE FINALE : {mae:.2f}")

fig = px.line(title=f"Modelisation Champion (MAE: {mae:.2f})")
fig.add_scatter(x=y_test.index, y=y_test, name="Reel", line=dict(color="#1a3a5f", width=3))
fig.add_scatter(x=y_test.index, y=preds, name="Prediction", line=dict(color="#c8102e", dash="dash"))
fig.update_layout(template="plotly_dark")
fig.show()